In [1]:
import csv

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

In [2]:
# 単語ベクトル化をGridSearchCVで使うのためのクラス
class myVectorizer(BaseEstimator, TransformerMixin):
    def __init__(self, method='tfidf', min_df=0.0005, max_df=0.10):
        self.method = method
        self.min_df = min_df
        self.max_df = max_df

    def fit(self, x, y=None):
        if self.method == 'tfidf':
            self.vectorizer = TfidfVectorizer(min_df=self.min_df, max_df=self.max_df)
        else:
            self.vectorizer = CountVectorizer(min_df=self.min_df, max_df=self.max_df)
        self.vectorizer.fit(x)
        return self

    def transform(self, x, y=None):
        return self.vectorizer.transform(x)

In [3]:
# GridSearchCV用パラメータ
PARAMETERS = [
    {
        'vectorizer__method':['tfidf', 'count'], 
        'vectorizer__min_df': [0.0003, 0.0004], 
        'vectorizer__max_df': [0.07, 0.10], 
        'classifier__C': [1, 3],    #10も試したが遅いだけでSCORE低い
        'classifier__solver': ['newton-cg', 'liblinear']},
    ]

# 時間がかかるので下記を省略
#        'vectorizer__min_df': [0.0003, 0.0004, 0.0005, 0.0006], 
#        'classifier__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']},

In [4]:
# ファイル読込
def read_csv_column(col):
    with open('./sentiment_stem.txt') as file:
        reader = csv.reader(file, delimiter='\t')
        header = next(reader)
        return [row[col] for row in reader]    

In [5]:
x_all = read_csv_column(1)
y_all = read_csv_column(0)
x_train, x_test, y_train, y_test = train_test_split(x_all, y_all)

In [6]:
def train(x_train, y_train, file):
    pipline = Pipeline([('vectorizer', myVectorizer()), ('classifier', LogisticRegression())])
    
    # clf は classificationの略
    clf = GridSearchCV(
            pipline, # 
            PARAMETERS,           # 最適化したいパラメータセット 
            cv = 5)               # 交差検定の回数
    
    clf.fit(x_train, y_train)
    pd.DataFrame.from_dict(clf.cv_results_).to_csv(file)
    
    print('Grid Search Best parameters:', clf.best_params_)
    print('Grid Search Best validation score:', clf.best_score_)
    print('Grid Search Best training score:', clf.best_estimator_.score(x_train, y_train))
    
    return clf.best_estimator_

In [7]:
def validate(estimator, x_test, y_test):
    
    for i, (x, y) in enumerate(zip(x_test, y_test)):
        y_pred = estimator.predict_proba([x])
        if y == np.argmax(y_pred).astype( str ):
            if y == '1':
                result = 'TP:正解がPositiveで予測もPositive'
            else:
                result = 'TN:正解がNegativeで予測もNegative'
        else:
            if y == '1':
                result = 'FN:正解がPositiveで予測はNegative'
            else:
                result = 'FP:正解がNegativeで予測はPositive'
        print(result, y_pred, x)
        if i == 29:
            break

In [8]:
%%time
estimator = train(x_train, y_train, 'gs_result.csv')

Grid Search Best parameters: {'classifier__C': 3, 'classifier__solver': 'newton-cg', 'vectorizer__max_df': 0.07, 'vectorizer__method': 'tfidf', 'vectorizer__min_df': 0.0003}
Grid Search Best validation score: 0.746248124062031
Grid Search Best training score: 0.896448224112056
CPU times: user 46.5 s, sys: 487 ms, total: 47 s
Wall time: 48.7 s


In [9]:
%%time
validate(estimator, x_test, y_test)

FN:正解がPositiveで予測はNegative [[0.68516093 0.31483907]] star colleg kid subject matter adult get temptat flesh unleash slightli craze overtli determin young woman night swim turn ocean troubl
TP:正解がPositiveで予測もPositive [[0.41581474 0.58418526]] find way tell simpl stori perhap simplest stori way seem compel even origin
TN:正解がNegativeで予測もNegative [[0.74596613 0.25403387]] film place realli dabbl around never gain much momentum
TN:正解がNegativeで予測もNegative [[0.72326093 0.27673907]] find grow frustrat detach vincent becom abhorr
FN:正解がPositiveで予測はNegative [[0.72997442 0.27002558]] emperor new cloth begin simpl plan well least plan
TN:正解がNegativeで予測もNegative [[0.78781904 0.21218096]] stitch bad manner ugli destruct littl cute factor mind ugli problem charact loveabl otherwis
TP:正解がPositiveで予測もPositive [[0.3774083 0.6225917]] make sever adapt writer work armenian canadian director atom egoyan broach origin treatment deepli person subject
TN:正解がNegativeで予測もNegative [[0.84168067 0.15831933]] stupo